# **Explore Different Prompting Techniques:**

* Without Prompt,
* Zero-Shot Prompt,
* One-Shot Prompt,
* Few-Shot Prompt,
* Chain-of-Thought Prompt,
* Tree-of-Thought Prompt,
* Role-Based Prompting,
* N-Shot Prompt

In [1]:
from dotenv import load_dotenv
import os


load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [52]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from IPython.display import display, Markdown


In [26]:
def load_llm(model_name:str="gpt-4o", temperature: float=0.2, max_tokens:int=2000) -> OpenAI:
    llm = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens=max_tokens)
    return llm

llm = load_llm(model_name="gpt-4o", temperature=0.8, max_tokens=256)

## **1. Without Prompt:**

In [7]:
# Example 01:

res = llm.invoke("Tell me something about India.")
display(Markdown(res.content))

India is a diverse and culturally rich country located in South Asia. It is the seventh-largest country in the world by land area and the second-most populous, with over 1.4 billion people. India is known for its vast array of languages, religions, and traditions, making it one of the most culturally diverse countries globally.

**Geography and Climate**: India has a varied landscape that includes the Himalayan mountain range in the north, fertile plains, deserts in the west, and coastal regions in the south. The climate also varies widely, from the tropical south to the temperate north.

**History**: India has a long and complex history that dates back to prehistoric times. It is the birthplace of major religions such as Hinduism, Buddhism, Jainism, and Sikhism. India was a major center for trade and culture throughout ancient and medieval times. It was colonized by the British Empire in the 19th century and gained independence in 1947.

**Culture**: Indian culture is renowned for its diversity and richness. It has a thriving arts scene, including music, dance, and cinema, with Bollywood being one of the largest film industries in the world. Indian cuisine is famous globally, known for its use of spices and flavors.

**Politics and Economy

In [23]:
# Example 02:

prompt = HumanMessage(content="""
You are responsible for creating a new software product. 
Now, whatever question is asked by the user to build the product, 
please answer in a way that the user can easily understand. 
Do not answer questions that are not related to building a software product.

Now, here is the user's question:
""")

query = "Tell me something about India"

# Merge the prompt and query
final_input = prompt.content + query
final_input


"\nYou are responsible for creating a new software product. \nNow, whatever question is asked by the user to build the product, \nplease answer in a way that the user can easily understand. \nDo not answer questions that are not related to building a software product.\n\nNow, here is the user's question:\nTell me something about India"

In [24]:
res = llm.invoke(final_input)
display(Markdown(res.content))

I'm here to help with questions related to building a software product. If you have any questions about software development, design, or implementation, feel free to ask!

## **2. Zero-Shot Prompt:**

**This involves giving the AI a task without any prior examples. You describe what you want in detail, assuming the AI has no prior knowledge of the task**. <br>

Zero-shot prompting is a method in natural language processing (NLP) where a language model is given a task without any prior training or fine-tuning on task-specific examples. The model is expected to perform the task based solely on its pre-trained knowledge and the information provided in the prompt. <br>

Zero-shot prompting relies on the model's ability to generalize from its training data to handle new, unseen tasks or questions. **The prompt must be designed carefully to provide sufficient context and clarity so that the model understands the task**.

In [27]:
template = """
You are a highly intelligent assistant that can summarize text effectively.
Please summarize the following text:

{text}

Summary:
"""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm | StrOutputParser()

query = """
This involves giving the AI a task without any prior examples. You describe what you want in detail,
assuming the AI has no prior knowledge of the task.

Zero-shot prompting is a method in natural language processing (NLP) where a language model is given a
task without any prior training or fine-tuning on task-specific examples. The model is expected to
perform the task based solely on its pre-trained knowledge and the information provided in the prompt.

Zero-shot prompting relies on the model's ability to generalize from its training data to handle new,
unseen tasks or questions. The prompt must be designed carefully to provide sufficient context and
clarity so that the model understands the task.
"""


response = chain.invoke(query)
display(Markdown(response))

Zero-shot prompting in natural language processing involves assigning a task to an AI model without prior examples or specific training on that task. The model is expected to complete the task using its pre-existing knowledge and the details provided in the prompt. This method relies on the model's ability to generalize from its prior training to address new and unfamiliar tasks, requiring well-crafted prompts for clarity and context.

## **3. One-Shot Prompt:**

<u>**You provide one example along with your prompt**. This helps the AI understand the context or format you’re expecting.</u> <br>

One-shot prompting involves providing the model with a single example of the task before asking it to perform a similar task. This technique gives the model a concrete example to follow, improving the quality of its output compared to zero-shot prompting.

In [34]:
template = """You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.
Your tasks is to translate the sentence from English to Bengali only.\n\n

Example:
English: How are you?
Bengali: কেমন আছো

English: {english}
Bengali:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['english']
)

prompt

PromptTemplate(input_variables=['english'], input_types={}, partial_variables={}, template='You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.\nYour tasks is to translate the sentence from English to Bengali only.\n\n\n\nExample:\nEnglish: How are you?\nBengali: কেমন আছো\n\nEnglish: {english}\nBengali:\n')

In [35]:
chain = prompt | llm | StrOutputParser()

response = chain.invoke("Tell me something about yourself.")
response

'আমার সম্পর্কে কিছু বলো।'

## **4. Few-Shot Prompt:**



In [53]:
few_shot_examples = [
    {"input": "What is an autonomous agent in LLMs?", "output": "Autonomous agents are systems that independently perform tasks using an LLM as their reasoning core, often involving planning, action execution, and reflection."},
    {"input": "What is the role of a planner in an agent?", "output": "The planner generates a sequence of actions to achieve a specific goal."},
    {"input": "How does the ReAct framework work in LLM-based agents?", "output": "ReAct combines reasoning and action to allow agents to solve problems and retrieve necessary information iteratively."},
    {"input": "What is the benefit of reflection in agents?", "output": "Reflection improves an agent’s future decisions by analyzing past actions and outcomes."},
    {"input": "Define a task decomposition approach in LLM agents.", "output": "Task decomposition breaks complex tasks into smaller subtasks that agents can execute sequentially or concurrently."},
    {"input": "What is the significance of tools in agent-based systems?", "output": "Tools provide external capabilities like APIs, calculators, or databases for agents to interact with the real world."},
    {"input": "What are multi-agent systems?", "output": "Multi-agent systems involve several agents collaborating to solve a task, often simulating teamwork."},
    {"input": "How do self-improvement mechanisms help LLM agents?", "output": "Agents refine their strategies by learning from successes and failures over multiple iterations."},
    {"input": "Explain the PEA architecture for agents.", "output": "PEA stands for Plan, Execute, and Assess, which outlines a structured agent workflow."},
    {"input": "How do LLM agents handle ambiguous tasks?", "output": "They use iterative clarification, asking follow-up questions or retrieving additional context to resolve ambiguity."},
]

# Few Shot Template:
few_shot_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

# Few Shot Prompt:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=few_shot_template,
    examples=few_shot_examples,
)


system_prompt = (
    "You are an AI Tutor, named 'Lily' specializing in Generative AI for K-12 students."
    "This AI Tutor Designed by Dibyendu Biswas, an AI/ML Engineer"
    "You provide clear, step-by-step explanations and encourage students to learn. "
    "If a question is unclear, ask for clarification."
    "Use four sentences maximum and keep the answer concise."
)


final_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    few_shot_prompt,
    ("user", "{input}"),
])


final_prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an AI Tutor, named 'Lily' specializing in Generative AI for K-12 students.This AI Tutor Designed by Dibyendu Biswas, an AI/ML EngineerYou provide clear, step-by-step explanations and encourage students to learn. If a question is unclear, ask for clarification.Use four sentences maximum and keep the answer concise."), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'What is an autonomous agent in LLMs?', 'output': 'Autonomous agents are systems that independently perform tasks using an LLM as their reasoning core, often involving planning, action execution, and reflection.'}, {'input': 'What is the role of a planner in an agent?', 'output': 'The planner generates a sequence of actions to achieve a specific goal.'}, {'input': 'How does the ReAc

In [55]:
chain = final_prompt | llm | StrOutputParser()

display(Markdown(chain.invoke("What is Few-Shot Prompting?")))

Few-shot prompting involves providing an LLM with a few examples within the prompt to guide its response or task performance.

## **5. Chain-of-Thought Prompt:**

**Here, you ask the AI to detail its thought process step-by-step. This is particularly useful for complex reasoning tasks.**


Chain-of-Thought (CoT) prompting involves guiding the model **to generate intermediate reasoning steps before arriving at a final answer**. This approach helps in complex tasks where a direct answer might not capture the entire reasoning process.

In [39]:
template = """You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.
Your tasks is to answers the user's query in below following format or helps users solve problems step by step:

Query: Your warehouse has 5 pallets of widgets. You purchase 2 more shipments of widgets. \
Each shipment contains 3 pallets. How many pallets of widgets do you have now?
Answer: The warehouse started with 5 pallets of widgets. 2 shipments of 3 pallets each is 6 pallets. \
5 pallets + 6 pallets = 11 pallets. The answer is 11 pallets.

Query: {question}


Let's think step by step.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.\nYour tasks is to answers the user's query in below following format or helps users solve problems step by step:\n\nQuery: Your warehouse has 5 pallets of widgets. You purchase 2 more shipments of widgets. Each shipment contains 3 pallets. How many pallets of widgets do you have now?\nAnswer: The warehouse started with 5 pallets of widgets. 2 shipments of 3 pallets each is 6 pallets. 5 pallets + 6 pallets = 11 pallets. The answer is 11 pallets.\n\nQuery: {question}\n\n\nLet's think step by step.\n")

In [41]:
chain = prompt | llm | StrOutputParser()

response = chain.invoke("Your finance department has $20,000 in the budget for a marketing campaign. I further allocate $20,000 for a marketing campaign and add $6,000 from other savings. How much is left in the budget?")

display(Markdown(response))

To determine how much is left in the budget, let's analyze the given information step by step:

1. The initial budget for the marketing campaign is $20,000.
2. An additional $20,000 is allocated for the marketing campaign, increasing the total budget. 
3. Another $6,000 is added from other savings.

Now, let's calculate the total budget:

- Start with the initial $20,000.
- Add the additional $20,000 allocated: \( 20,000 + 20,000 = 40,000 \).
- Add the $6,000 from other savings: \( 40,000 + 6,000 = 46,000 \).

Therefore, the total amount in the budget for the marketing campaign is $46,000.

The answer is $46,000 in the budget.

In [45]:
cot_prompt = PromptTemplate.from_template("""
You are a helpful AI Assistant named Lily, who helps users solve problems step by step.

Question: {question}

Let's think step by step.
""")

# Step 2: Add the LLM and Output Parser
parser = StrOutputParser()

# Step 3: Chain everything
chain = cot_prompt | llm | parser

# Step 4: Invoke with a reasoning-based question
response = chain.invoke("If a train travels 60 km in 1.5 hours, what is its average speed in km/h?")

display(Markdown(response))

To find the average speed of the train, we can use the formula for average speed, which is:

\[
\text{Average speed} = \frac{\text{Total distance}}{\text{Total time}}
\]

Let's break down the problem step by step:

1. **Determine the Total Distance:**
   - The train travels a total distance of 60 kilometers.

2. **Determine the Total Time:**
   - The total time taken by the train is 1.5 hours.

3. **Apply the Formula:**
   - Substitute the total distance and total time into the formula to find the average speed:
   \[
   \text{Average speed} = \frac{60 \text{ km}}{1.5 \text{ hours}}
   \]

4. **Calculate the Average Speed:**
   - Perform the division:
   \[
   \text{Average speed} = 40 \text{ km/h}
   \]

Thus, the average speed of the train is 40 kilometers per hour.

## **6. Tree-of-Thought Prompt:**

**Tree-of-Thought (ToT) prompting expands on CoT by exploring multiple reasoning paths in parallel**, akin to a decision tree. This technique is useful for tasks that can have multiple **plausible approaches** or solutions.

In [46]:
prompt = PromptTemplate.from_template("""
You are Lily, a thoughtful assistant. Consider multiple possible approaches before solving.

Question: {question}

Think of at least two different ways to solve this. Then, choose the best solution.

Option 1:
...

Option 2:
...

Best Answer:
""")

prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nYou are Lily, a thoughtful assistant. Consider multiple possible approaches before solving.\n\nQuestion: {question}\n\nThink of at least two different ways to solve this. Then, choose the best solution.\n\nOption 1:\n...\n\nOption 2:\n...\n\nBest Answer:\n')

In [47]:
chain = prompt | llm | StrOutputParser()

display(Markdown(chain.invoke("How can we reduce traffic in large Indian cities?")))

To address the issue of traffic congestion in large Indian cities, we need to consider various approaches that can be tailored to the unique challenges of urban environments in India. Here are two potential solutions:

Option 1: Enhance Public Transportation Systems
- **Develop Integrated Transport Networks**: Improve and expand the existing public transportation infrastructure, including buses, metro, and commuter trains, to provide more reliable and frequent services. Integrating these systems can create a seamless travel experience.
- **Upgrade Technology**: Implement smart technologies like real-time tracking apps, digital payment systems, and efficient scheduling to make public transportation more user-friendly and efficient.
- **Encourage Use through Incentives**: Offer incentives for using public transport, such as discounted fare passes or tax benefits, to make it a more attractive option compared to driving personal vehicles.

Option 2: Implement Traffic Management and Control Measures
- **Smart Traffic Signals**: Use AI-driven traffic signals that adapt to real-time traffic conditions, improving flow and reducing congestion at peak times.
- **Carpool and Ride-Sharing Promotions**: Promote carpooling and ride-sharing services through dedicated lanes or reduced toll charges, thereby reducing the number of vehicles on the road.
- **Congestion Pricing**: Introduce congestion pricing in highly congested zones

## **7. Role Based Prompting:**

Use persona-based **instructions for behavior control**. **Great for tone, context, or expert emulation**.

In [49]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional Indian History professor at a university."),
    ("user", "{query}")
])

prompt

ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a professional Indian History professor at a university.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

In [50]:
chain = prompt | llm | StrOutputParser()

display(Markdown(chain.invoke("Can you explain the importance of the Battle of Plassey?")))

Certainly! The Battle of Plassey, fought on June 23, 1757, was a pivotal confrontation between the British East India Company led by Robert Clive and the Nawab of Bengal, Siraj-ud-Daulah. This battle marked a significant turning point in Indian history due to several key reasons:

1. **Establishment of British Power**: The victory at Plassey laid the foundation for British colonial rule in India. It marked the beginning of British political dominance in India, eventually leading to the establishment of British rule over the entire subcontinent.

2. **Control over Bengal**: Bengal was one of the most prosperous and resource-rich regions in India at the time. The British gained control over Bengal's revenues, enabling the East India Company to finance its expansion and solidify its presence in India.

3. **Economic Exploitation**: The control over Bengal's resources facilitated the economic exploitation of the region. The wealth extracted from Bengal contributed significantly to Britain's economic growth and its ability to finance further colonial ventures.

4. **Political Strategy and Alliances**: The battle highlighted the use of political strategies and alliances in military conflicts. The British were able to win the battle, in part, due to their alliance with key Indian figures such as

In [51]:
display(Markdown(chain.invoke("What is Chain of Thought (CoT) prompting?")))

Chain of Thought (CoT) prompting is a technique used in natural language processing and artificial intelligence to improve the reasoning and decision-making capabilities of AI models. It involves guiding the AI through a series of intermediate steps or reasoning processes to arrive at a solution or conclusion. By breaking down a complex problem into smaller, more manageable parts, CoT prompting helps models generate more accurate and coherent responses.

In practice, Chain of Thought prompting might involve explicitly instructing the model to consider various aspects of a problem sequentially, much like a human would go through a logical reasoning process. This can be particularly useful in tasks that require multi-step reasoning, such as mathematical problem-solving, complex question answering, and more nuanced decision-making scenarios. By encouraging a structured approach to problem-solving, CoT prompting aims to enhance the AI's ability to handle complex queries more effectively.

## **8. N-Shot Prompting:**

Similart to Few-Shot Prompt with n-examples.



## **Links:**

https://github.com/dibyendubiswas1998/Generative-AI/tree/main/Prompt%20Engineering